In [18]:
# 딥러닝 및 NLP 라이브러리 설치
# (이미 설치되어 있다면 이 셀은 건너뛰어도 됩니다)
!pip install torch transformers scikit-learn pandas tqdm


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import torch
from torch.utils.data import Dataset, DataLoader
# 1. 여기서 AdamW를 지웁니다.
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# 2. 대신 여기서 AdamW를 가져옵니다. (PyTorch 내장 최적화 함수 사용)
from torch.optim import AdamW

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"현재 사용 장치: {device}")

# 경로 설정
DATA_PATH ="../data/03_final/train_data.csv"
MODEL_SAVE_PATH = "../models/koelectra_finetuned/"

os.makedirs(MODEL_SAVE_PATH, exist_ok=True)

현재 사용 장치: cuda


In [20]:
!pip install openpyxl
import pandas as pd
import os

# 파일 경로
ORIGINAL_PATH = "../data/03_final/train_data.csv"  # 현재 깨지는 파일
FIXED_PATH = "../data/03_final/train_data_fixed.csv" # 새로 만들 진짜 CSV

print(f"📂 파일 복구 및 로드 시도: {ORIGINAL_PATH}")

try:
    # 1. 확장자는 .csv지만 실제론 엑셀 파일이므로 'read_excel'로 읽습니다.
    df = pd.read_excel(ORIGINAL_PATH, engine='openpyxl')
    print("✅ 엑셀 형식으로 읽기 성공!")

    # 2. 읽은 데이터를 '진짜 CSV'로 다시 저장합니다 (나중을 위해)
    df.to_csv(FIXED_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 정상적인 CSV 파일로 변환 완료: {FIXED_PATH}")

    # 3. 데이터 확인
    print(f"로드된 데이터 개수: {len(df)}건")
    print(f"컬럼 목록: {df.columns.tolist()}")
    display(df.head(3))

    # --- 전처리 및 데이터 분리 ---
    text_col = 'cleaned_review' if 'cleaned_review' in df.columns else 'review'

    if text_col in df.columns and 'label' in df.columns:
        df = df[[text_col, 'label']].dropna()
        df['label'] = pd.to_numeric(df['label'], errors='coerce')
        df = df.dropna(subset=['label'])
        df['label'] = df['label'].astype(int)

        from sklearn.model_selection import train_test_split
        train_texts, val_texts, train_labels, val_labels = train_test_split(
            df[text_col].tolist(),
            df['label'].tolist(),
            test_size=0.2,
            random_state=42,
            stratify=df['label']
        )
        print("-" * 30)
        print(f"학습 데이터: {len(train_texts)}개, 검증 데이터: {len(val_texts)}개")
        print("준비 완료! 다음 셀([Cell 4])로 넘어가세요.")
    else:
        print(f"❌ 오류: '{text_col}' 또는 'label' 컬럼이 없습니다. 컬럼명을 확인해주세요.")

except Exception as e:
    print(f"❌ 오류 발생: {e}")
    print("팁: '!pip install openpyxl'을 먼저 실행했는지 확인해주세요.")


  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]




[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ 엑셀 형식으로 읽기 성공!
✅ 정상적인 CSV 파일로 변환 완료: ../data/03_final/train_data_fixed.csv
로드된 데이터 개수: 2500건
컬럼 목록: ['Unnamed: 0', 'product_name', 'rating', 'skin_type', 'review', 'cleaned_review', 'id', 'label']


,Unnamed: 0,product_name,rating,skin_type,review,cleaned_review,id,label
0,119258,구달 청귤 비타C 크림 50ml (아이크림15ml 증정),5,복합성,구달의 다른 제품을 썼을때 향도 좋고 미백효과도 좀 본것같아서 구매했습니다. 이 제...,구달의 다른 제품을 썼을때 향도 좋고 미백효과도 좀 본것같아서 구매했습니다. 이 제...,0,2
1,75013,아벤느 똘레랑스 컨트롤 크림 40ml 기획,5,건성,아벤느 제품은 프랑스 약국 브랜드 제품으로 워낙 유명해서 한번 써보려고 했는데 좋은...,아벤느 제품은 프랑스 약국 브랜드 제품으로 워낙 유명해서 한번 써보려고 했는데 좋은...,1,0
2,97775,리얼베리어 인텐스 모이스처 크림 기획 (본품50ml+20ml+폼50ml 증정),5,NaN,몇년째 꾸준하게 쓰고 있어요 이거보다 나은건 아직 못 찾았네요.. 올리브영에서 사면...,몇년째 꾸준하게 쓰고 있어요 이거보다 나은건 아직 못 찾았네요.. 올리브영에서 사면...,2,4


------------------------------
학습 데이터: 2000개, 검증 데이터: 500개
준비 완료! 다음 셀([Cell 4])로 넘어가세요.


In [21]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW  # ✅ 수정됨: PyTorch 내장 AdamW 사용
import numpy as np
from tqdm.notebook import tqdm

# GPU 설정 (없으면 CPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔥 현재 사용 장치: {device}")

# 토크나이저 로드 (KoELECTRA)
MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 데이터셋 클래스 정의
class ReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 데이터로더 생성
# (train_texts, val_texts 등은 방금 전 단계에서 이미 만들어져 있습니다)
train_dataset = ReviewDataset(train_texts, train_labels, tokenizer)
val_dataset = ReviewDataset(val_texts, val_labels, tokenizer)

BATCH_SIZE = 16
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

print("✅ 데이터셋 준비 완료!")

🔥 현재 사용 장치: cuda
✅ 데이터셋 준비 완료!


In [23]:
# 5개의 라벨(0~4)로 분류하는 모델 로드
# 🚨 weights_only=False 옵션을 추가하여 버전 오류를 우회합니다.
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=5,
    weights_only=False
)
model = model.to(device)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=2e-5)

# 정확도 계산 함수
def calc_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

print("✅ 모델 로드 완료!")

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ 모델 로드 완료!


In [24]:
# ---------------------------------------------------------
# 🚨 수정 사항: tqdm.notebook 대신 일반 tqdm을 사용하여 에러 방지
from tqdm import tqdm
# ---------------------------------------------------------

EPOCHS = 3

print(f"🔥 학습 시작! (GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Unknown'})")
print(f"총 {EPOCHS} Epoch 동안 학습합니다.\n")

for epoch in range(EPOCHS):
    print(f"======== Epoch {epoch + 1} / {EPOCHS} ========")

    # [학습 모드]
    model.train()
    total_loss = 0
    train_acc = 0

    # tqdm()으로 감싸서 진행률 표시
    for batch in tqdm(train_loader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        model.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        # 정확도 누적
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()
        train_acc += calc_accuracy(logits, label_ids)

    avg_train_loss = total_loss / len(train_loader)
    avg_train_acc = train_acc / len(train_loader)

    print(f"  ✅ Average Train Loss: {avg_train_loss:.4f}")
    print(f"  ✅ Average Train Accuracy: {avg_train_acc:.4f}")

    # [검증 모드]
    model.eval()
    val_acc = 0
    val_loss = 0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

            logits = outputs.logits.detach().cpu().numpy()
            label_ids = labels.to('cpu').numpy()
            val_acc += calc_accuracy(logits, label_ids)

    avg_val_loss = val_loss / len(val_loader)
    avg_val_acc = val_acc / len(val_loader)

    print(f"  🔹 Validation Loss: {avg_val_loss:.4f}")
    print(f"  🔹 Validation Accuracy: {avg_val_acc:.4f}")
    print("-" * 50)

print("\n🎉 모든 학습이 완료되었습니다!")

🔥 학습 시작! (GPU: NVIDIA GeForce RTX 4070)
총 3 Epoch 동안 학습합니다.

======== Epoch 1 / 3 ========


Training: 100%|██████████| 125/125 [00:13<00:00,  9.26it/s]


  ✅ Average Train Loss: 1.3339
  ✅ Average Train Accuracy: 0.4355


Validation: 100%|██████████| 32/32 [00:01<00:00, 30.16it/s]


  🔹 Validation Loss: 1.0890
  🔹 Validation Accuracy: 0.6387
--------------------------------------------------
======== Epoch 2 / 3 ========


...

  🔹 Validation Loss: 0.7000
  🔹 Validation Accuracy: 0.7695
--------------------------------------------------
======== Epoch 3 / 3 ========


Training: 100%|██████████| 125/125 [00:13<00:00,  9.37it/s]


  ✅ Average Train Loss: 0.6122
  ✅ Average Train Accuracy: 0.7905


Validation: 100%|██████████| 32/32 [00:01<00:00, 31.77it/s]

  🔹 Validation Loss: 0.5684
  🔹 Validation Accuracy: 0.8105
--------------------------------------------------

🎉 모든 학습이 완료되었습니다!


In [25]:
# 학습된 모델 저장
save_path = "../models/koelectra_finetuned/"
os.makedirs(save_path, exist_ok=True)

# 모델 가중치 저장
torch.save(model.state_dict(), os.path.join(save_path, "koelectra_model.pt"))
print(f"✅ 모델 저장 완료: {save_path}koelectra_model.pt")

✅ 모델 저장 완료: models/koelectra_finetuned/koelectra_model.pt
